<br> Ayman FAHSI | A20440820

Mouhammad BAZZI | A20522180


CS577 - Fall 2022</br> <h1><br><b><font color='red'>Project</font></br></h1>

# <h1><b><font color='ORANGE'>MAIN PROGRAM</h1>

## **LIBRAIRES IMPORTATIONS**

In [ ]:
#Importing the libraries
import cv2
import numpy as np
from tensorflow.keras.models import load_model

## **GLOBAL VARIABLES**

In [ ]:
############################################################################################################
#WE DEFINE THE CONSTANTS

GENDER_MODEL_PATH = "../data/models/gender/gender_residual_aug.h5"
EMOTION_MODEL_PATH = "../data/models/emotion/emotion_residual_noaug.h5"

IS_WEBCAM_INPUT = False  #If we want to use the webcam as input
INPUT_VIDEO_PATH = "../data/video_for_testing.mp4" #Path of the video if we want to use a video as input

############################################################################################################
############################################################################################################

## **MAIN PROGRAM**

In [ ]:
#We will use OpenCV to detect the faces, and then apply our models to each face to predict there gender and emotion
#We will use the following emotions: ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

#--------------------------------------------#

EMOTIONS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

#--------------------------------------------#

#We will use the Haar Cascade Classifier to detect the faces
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#We will use our gender model to predict the gender
gender_model = load_model(GENDER_MODEL_PATH)

#We will use our emotion model to predict the emotion
emotion_model = load_model(EMOTION_MODEL_PATH)

#--------------------------------------------#

#If we want to use the webcam as input
if IS_WEBCAM_INPUT:
    cap = cv2.VideoCapture(0)
#If we want to use a video as input
else:
    cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

#--------------------------------------------#

#We will loop over the frames
while True:
    #Read the frame
    _, img = cap.read()
    # Convert to color image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #Detect the faces in color image
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    if len(faces) > 0:
        #We will apply the gender model to each face
        #Make the face 10% bigger
        for (x, y, w, h) in faces:
            #We will make the face 10% bigger
            x = int(x - 0.1 * w)
            y = int(y - 0.1 * h)
            w = int(1.2 * w)
            h = int(1.2 * h)
            #We will make sure that the face is not out of the image
            if x < 0:
                x = 0
            if y < 0:
                y = 0
            if x + w > img.shape[1]:
                w = img.shape[1] - x
            if y + h > img.shape[0]:
                h = img.shape[0] - y


            #Crop the face
            face = gray[y:y+h, x:x+w]
            color_face = img[y:y+h, x:x+w]

            #--------------------------------------------#

            #Resize the face to 64x64
            color_face = cv2.resize(color_face, (64, 64))
            #Reshape the face to 1x64x64
            face_g = color_face.reshape(1, 64, 64, 3)
            #Predict the gender
            prediction_g = gender_model.predict(face_g)
            #Get the index of the highest probability
            index_g = np.argmax(prediction_g)

            #Draw the rectangle around the face
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
            #Draw the text
            if index_g == 0:
                cv2.putText(img, "Woman", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(img, "Man", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            #--------------------------------------------#

            #Resize the face to 48x48
            face_e = cv2.resize(face, (48, 48))
            #Reshape the face to 1x48x48
            face_e = face_e.reshape(1, 48, 48, 1)
            #Predict the emotion
            prediction_e = emotion_model.predict(face_e)
            #Get the index of the highest probability
            index_e = np.argmax(prediction_e)
            #Draw the text
            cv2.putText(img, EMOTIONS[index_e], (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            #--------------------------------------------#

    #Display
    cv2.imshow('img', img)

    #Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
    
#Release the VideoCapture object
cap.release()


